In [1]:
import os
import pandas as pd
import pathlib

AIRFLOW_PATH = os.path.normpath(os.getcwd() + '../../airflow')

In [2]:
df = pd.read_pickle(f'{AIRFLOW_PATH}/datastore/wikipedia_result.pkl')

In [62]:
# Explode tuples
genres_full = df['Genre'].explode('Genre')
# Split any remaining comma seperated lists
genres_full = genres_full.str.split(',', expand=True)
genres_full = genres_full[0].to_frame(name="Genre").merge(genres_full[1].to_frame(name="Genre"), how='outer', on='Genre')
genres_full = genres_full['Genre']
genres_full = genres_full[genres_full.notnull()]

# Capitalize words
genres_capitalized = genres_full.map(lambda a: a.title())

# Reduce to unique entries
genres_unique = genres_capitalized.unique()
# unique gives us an ndarray, convert back to a dataframe
genres_unique = pd.DataFrame(data=genres_unique, columns=['Genre'])

# Sort
genres_unique.sort_values(by='Genre')



,Genre
221,Action
241,Action Role-Playing
213,Adventure
212,Art
220,Beat 'Em Up
...,...
200,Vehicle Simulation Game
52,Vehicular Combat
189,Vehicular Sandbox
16,Visual Novel


In [86]:
def generate_genre_map(df):
    """Create a base mapping to a standardized genre name from the mess that Wikipedia has.

    This function takes a DataFrame (df) from the Wikipedia job.
    """
    # Explode tuples
    genres_full = df['Genre'].explode('Genre')
    # Split any remaining comma seperated lists
    genres_full = genres_full.str.split(',', expand=True)
    genres_full = genres_full[0].to_frame(name="Genre").merge(genres_full[1].to_frame(name="Genre"), how='outer', on='Genre')
    genres_full = genres_full['Genre']
    genres_full = genres_full[genres_full.notnull()]

    # Create the mapping dataframe
    genre_map = pd.DataFrame({
        'from': genres_full,
        'to': genres_full
    })

    # Capitalize words
    genre_map['to'] = genre_map['to'].map(lambda a: a.title())

    # Trim spaces
    genre_map['to'] = genre_map['to'].map(lambda a: a.trim())

    # Reduce to unique entries
    genre_map = genre_map.drop_duplicates(subset=['from'])

    genre_map = genre_map.sort_values('to')

    return genre_map


In [89]:
out_df = generate_genre_map(df)
out_df.to_csv('./genre_mapping.csv', index=True)
# out_d
